#Day 3

Welcome to Day 3-The Final Day. Today we will go over convolutions, noise, and denoising.

##### Convolutions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import data

#this is the package we will be using today to convolve images, but there are other commonly used ones as well
from scipy.ndimage import convolve

In [ ]:
#lets start by loading in our image - our favorite cat Chelsea
image = data.chelsea()
image = image/255
plt.imshow(image)
plt.show()

In [ ]:
#to start, lets look at a simple blur - like the first one in the 3Blue1Brown video
blurKernel = np.array([[1/9, 1/9, 1/9],
                       [1/9, 1/9, 1/9],
                       [1/9, 1/9, 1/9]
                       ])
#the kernel here is just the same value throughout, so that means that each new pixel will be an average of the 3x3 around it
convolvedImage = convolve(image, blurKernel, axes=[0,1])

plt.imshow(convolvedImage)
plt.show()

In [ ]:
#try it yourself, convolve Chelsea with a filter that creates an outline - if you don't know what the kernel should looklike, go back to the Setosa site and see what they use
outlineKernel = np.array([

                          ])

convolvedImage = convolve(image, outlineKernel, axes=[0,1])

convolvedImage = np.clip(convolvedImage, 0, 1) #this restricts the final values we get to be between 0 and 1

plt.imshow(convolvedImage)
plt.show()

Padding

In [ ]:
#for us right now, there isn't a big use case for padding, but lets still see what it would look like
zeroPaddedImage = np.pad(image,
                     pad_width=((2, 2), (2, 2), (0, 0)),
                     mode='constant',
                     constant_values=0)

plt.imshow(zeroPaddedImage)
plt.show()
#now there is a black border - a border of 0's - around the image

In [ ]:
replicationPaddedImage = np.pad(image,
                     pad_width=((2, 2), (2, 2), (0, 0)), #the reason we have to put this list-thingy is so the color dimension isn't also padded, don't worry about that too much though
                     mode='symmetric') #symmetric is the same as replication

plt.imshow(replicationPaddedImage)
plt.show()

In [ ]:
#run this cell to see a few different convolutions - try and reason through what the kernel in each is doing
def convAndShowImage(image, kernel):
    convolvedImage = convolve(image, kernel, axes=[0, 1])
    #clip the image
    convolvedImage = np.clip(convolvedImage, 0, 1)
    plt.imshow(convolvedImage)
    plt.show()


image = data.chelsea()
#normalize image
image = image / 255

edgeDetectionKernel = np.array([[-1, 0, 1],
                                [-2, 0, 2],
                                [-1, 0, 1]])

laplacianKernel = np.array([[0, -1, 0],
                            [-1, 5, -1],
                            [0, -1, 0]])
gaussianFilter = np.array([[1, 2, 1],
                         [2, 4, 2],
                         [1, 2, 1]]) / 16
embossKernel = np.array([[-2, -1, 0],
                         [-1, 1, 1],
                         [0, 1, 2]])

print("Original")
plt.imshow(image)
plt.show()
print("Edge Detection")
convAndShowImage(image, edgeDetectionKernel)
print("Sharpen")
convAndShowImage(image, laplacianKernel)
print("Gaussian Blur")
convAndShowImage(image, gaussianFilter)
print("Emboss")
convAndShowImage(image, embossKernel)




##### Noise

In [ ]:
#lets see what different types of noise look like

#here is the original image - no noise
originalImage = data.camera()
plt.imshow(originalImage, cmap="gray")
plt.show()

In [ ]:
#salt and pepper noise - don't worry about what this function is doing, but if you want to try and understand it, feel free to
def addSaltPepperNoise(image):
  #normalize image
  noiseImage = np.copy(image) / 255

  saltAmount = np.random.randint(1000, 20000)
  for i in range(saltAmount):
    x = np.random.randint(0, image.shape[0])
    y = np.random.randint(0, image.shape[1])
    noiseImage[x, y] = 1

  pepperAmount = np.random.randint(1000, 20000)
  for i in range(pepperAmount):
    x = np.random.randint(0, image.shape[0])
    y = np.random.randint(0, image.shape[1])
    noiseImage[x, y] = 0

  #go back to 255
  noiseImage = noiseImage * 255
  return noiseImage

print("Salt and Pepper Noise")
saltPepperNoiseImage = addSaltPepperNoise(originalImage)
plt.imshow(saltPepperNoiseImage, cmap="gray")
plt.show()

#poisson noise
print("Poisson Noise")
poissonNoise = np.random.poisson(lam=0.1, size=originalImage.shape)
poissonNoiseImage = originalImage/255 + poissonNoise
poissonNoiseImage = np.clip(poissonNoiseImage, 0, 1)
plt.imshow(poissonNoiseImage, cmap="gray")
plt.show()

#gaussian noise
print("Guassian Noise")
gaussianNoise = np.random.normal(loc=0, scale=0.1, size=originalImage.shape) #the loc is the same thing as the mean, and the scale is the same thing as the standard deviation
gaussianNoiseImage = originalImage/255 + gaussianNoise
gaussianNoiseImage = np.clip(gaussianNoiseImage, 0, 1)
plt.imshow(gaussianNoiseImage, cmap="gray")
plt.show()

In [ ]:
#let's see what Gaussian Noise looks like in RGB images
print("Original Image")
rgbImage = data.astronaut()
plt.imshow(rgbImage)
plt.show()

print("Gaussian Noise")
rgbImage = rgbImage/255
noise = np.random.normal(loc=0, scale=0.3, size=rgbImage.shape)
gaussianNoiseImage = rgbImage + noise
gaussianNoiseImage = np.clip(gaussianNoiseImage, 0, 1)
plt.imshow(gaussianNoiseImage)
plt.show()

#try messing around with different scale values (standard deviation values) and see what happens - what happens when it is really low? what happens when it is really high?

For RGB images, each component of the color vector will be independtly changed based on the gaussian distribution. This means that both the red, green, and blue color channels have some gaussian noise.

##### Denoising

In [ ]:
print("Original Image")
plt.imshow(originalImage, cmap="gray")
plt.show()

print("Gaussian Noise")
plt.imshow(gaussianNoiseImage, cmap="gray")
plt.show()

In [ ]:
#Denoising the image with a simple blur - a uniform average kernel (all the kernel values are the same)
print("Denoising with Standard Blur")
blurKernel = np.array([[1/9, 1/9, 1/9],
                       [1/9, 1/9, 1/9],
                       [1/9, 1/9, 1/9]
                       ])
convolvedImage = convolve(gaussianNoiseImage, blurKernel, axes=[0,1])
plt.imshow(convolvedImage, cmap="gray")
plt.show()

In [ ]:
#denoising with a gaussian blur
gaussianBlurKernel = np.array([
    [1/16, 2/16, 1/16],
    [2/16, 4/16, 2/16],
    [1/16, 2/16, 1/16]
    ])
convolvedImage = convolve(gaussianNoiseImage, gaussianBlurKernel, axes=[0,1])
plt.imshow(convolvedImage, cmap="gray")
plt.show()
#take a second to think about why a gaussian blur will denoise better than a standard blur

Again, this type of denoising will never completely restore the original image, but it does accomplish its goal of lowering the amount of noise.

In [ ]:
#lets try it with color
print("Original Image")
rgbImage = data.chelsea() #back to Chelsea! sorry if you aren't a cat person
plt.imshow(rgbImage)
plt.show()

print("Noisy Image")
rgbImage = rgbImage/255
noise = np.random.normal(loc=0, scale=0.05, size=rgbImage.shape)
gaussianNoiseImage = rgbImage + noise
gaussianNoiseImage = np.clip(gaussianNoiseImage, 0, 1)
plt.imshow(gaussianNoiseImage)
plt.show()

In [ ]:
#try filling in the kernel and convolving it with the image to denoise the image
print("Denoising with Gaussian Blur")
denoiseKernel = np.array([


                          ])
denoisedImage = convolve(gaussianNoiseImage, denoiseKernel, axes=[0,1])
plt.imshow(denoisedImage)
plt.show()